In [1]:
import pandas as pd 

df = pd.read_csv("Total Load - Day Ahead _ Actual_202301010000-202401010000 (3).csv")


In [2]:
def convert_time(df, time_column):
    # Check if the time data represents an interval
    if " - " in df[time_column].iloc[0]:
        # Split the time interval into start and end times
        df["Start Time"] = df[time_column].str.split(
            " - ", expand=True
        )[0]
        #drop initial time column
        df.drop(time_column,inplace=True,axis=1)
        # create a new column with only the start time of Time column 
        df["Start Time"] = pd.to_datetime(df["Start Time"], format="%d.%m.%Y %H:%M")
        #assign to the column with the same initial name (time_column)
        df[time_column]=df['Start Time']
        #drop Start Time column 
        df.drop('Start Time',inplace=True,axis=1)
    else:
        # Convert the time data to datetime
        df[time_column] = pd.to_datetime(df[time_column], format="%d.%m.%Y %H:%M")

    return df

# Apply the function to your DataFrame
time_column='Time (CET/CEST)'
df_read = convert_time(df, time_column)


In [3]:
df_read.head()

,Day-ahead Total Load Forecast [MW] - Netherlands (NL),Actual Total Load [MW] - Netherlands (NL),Time (CET/CEST)
0,11330,9912,2023-01-01 00:00:00
1,11251,9813,2023-01-01 00:15:00
2,11154,9877,2023-01-01 00:30:00
3,11045,9861,2023-01-01 00:45:00
4,10929,9847,2023-01-01 01:00:00


In [4]:
time_resolution_minutes=5

# Set the 'time_column' as the index explicitly
df_read.set_index(time_column, inplace=True)

# Find the first row with '-'
first_invalid_row = df_read.eq('-').any(axis=1).idxmax()


In [6]:
#find the dataset frequency 
difference = df_read.index.to_series().diff()[1]
minutes_difference = difference.total_seconds() / 60



C:\Users\saeed.misaghian\AppData\Local\Temp\ipykernel_12540\1296338516.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  difference = df_read.index.to_series().diff()[1]


In [7]:
minutes_difference

15.0

In [8]:
# Keep only the rows until the row before the first_invalid_row
df_read = df_read.loc[:first_invalid_row - pd.Timedelta(minutes=minutes_difference)]



In [9]:
df_read

,Day-ahead Total Load Forecast [MW] - Netherlands (NL),Actual Total Load [MW] - Netherlands (NL)
Time (CET/CEST),,
2023-01-01 00:00:00,11330,9912
2023-01-01 00:15:00,11251,9813
2023-01-01 00:30:00,11154,9877
2023-01-01 00:45:00,11045,9861
2023-01-01 01:00:00,10929,9847
...,...,...
2023-11-05 12:30:00,9501,10087
2023-11-05 12:45:00,9420,10034
2023-11-05 13:00:00,9491,10102


In [10]:
# Check for duplicate index values
duplicates = df_read.index.duplicated()

# Handle duplicates (for example, by keeping the first occurrence and dropping the others)
df_read = df_read[~duplicates]
#be sure that columns are not object (we have already taken care of nan etc.. so we should have only numbers)
df_read = df_read.apply(pd.to_numeric, errors='coerce')

df_read.resample(f"{time_resolution_minutes}T").interpolate()

,Day-ahead Total Load Forecast [MW] - Netherlands (NL),Actual Total Load [MW] - Netherlands (NL)
Time (CET/CEST),,
2023-01-01 00:00:00,11330.000000,9912.000000
2023-01-01 00:05:00,11303.666667,9879.000000
2023-01-01 00:10:00,11277.333333,9846.000000
2023-01-01 00:15:00,11251.000000,9813.000000
2023-01-01 00:20:00,11218.666667,9834.333333
...,...,...
2023-11-05 13:10:00,9560.333333,10083.333333
2023-11-05 13:15:00,9595.000000,10074.000000
2023-11-05 13:20:00,9611.666667,10077.666667


In [68]:
def convert_time(df, time_column):
    """
    Converts a time column in a DataFrame to a consistent datetime format.

    Args:
        df (pd.DataFrame): The DataFrame containing the time data.
        time_column (str): The name of the time column to be converted.

    Returns:
        pd.DataFrame: The DataFrame with the time column converted to datetime.
    """

    if " - " in df[time_column].iloc[0]:
        # Split the time interval into start and end times
        df["Start Time"] = df[time_column].str.split(" - ", expand=True)[0]

        # Drop the original time column
        df.drop(time_column, inplace=True, axis=1)

        # Convert the "Start Time" to datetime
        df["Start Time"] = pd.to_datetime(df["Start Time"], format="%d.%m.%Y %H:%M")

        # Assign the datetime values to the original time column name
        df[time_column] = df["Start Time"]

        # Drop the "Start Time" column
        df.drop("Start Time", inplace=True, axis=1)
    else:
        # Convert the time data to datetime
        df[time_column] = pd.to_datetime(df[time_column], format="%d.%m.%Y %H:%M")

    return df

# Apply the function to your DataFrame
na_values = ["nan", "n/e", "no", "na"]
# Read the uploaded CSV file
df_read = pd.read_csv("Total Load - Day Ahead _ Actual_202201010000-202301010000.csv", na_values=na_values)

# Set the 'time_column' as the index explicitly
df_read.set_index(time_column, inplace=True)

if (df_read == "-").any().any():
    first_invalid_row = df_read.eq("-").any(axis=1).idxmax()    
else:
    skip_invalid_row='True'
         

for column in df_read.columns[1:]:
        df_read[column] = pd.to_numeric(df_read[column], errors="coerce")




df_read = df_read.interpolate() 


In [72]:
if (df_read == "-").any().any():
    print('yes')
else:
    print('no')    

no


In [69]:
first_invalid_row

'01.01.2022 00:00 - 01.01.2022 00:15'

In [63]:
df_read.reset_index(inplace=True)

In [71]:
df_read.tail()

,Day-ahead Total Load Forecast [MW] - Netherlands (NL),Actual Total Load [MW] - Netherlands (NL)
Time (CET/CEST),,
31.12.2022 22:45 - 31.12.2022 23:00,12107.0,10395.0
31.12.2022 23:00 - 31.12.2022 23:15,11956.0,10313.0
31.12.2022 23:15 - 31.12.2022 23:30,11817.0,10248.0
31.12.2022 23:30 - 31.12.2022 23:45,11661.0,10169.0
31.12.2022 23:45 - 01.01.2023 00:00,11476.0,10086.0


In [65]:

df_read = convert_time(df_read, time_column)



In [66]:
df_read

,Day-ahead Total Load Forecast [MW] - Netherlands (NL),Actual Total Load [MW] - Netherlands (NL),Time (CET/CEST)
0,11766.0,10446.0,2022-01-01 00:00:00
1,11608.0,10388.0,2022-01-01 00:15:00
2,11450.0,10340.0,2022-01-01 00:30:00
3,11305.0,10322.0,2022-01-01 00:45:00
4,11213.0,10326.0,2022-01-01 01:00:00
...,...,...,...
35039,12107.0,10395.0,2022-12-31 22:45:00
35040,11956.0,10313.0,2022-12-31 23:00:00
35041,11817.0,10248.0,2022-12-31 23:15:00
35042,11661.0,10169.0,2022-12-31 23:30:00


In [67]:
first_invalid_row.split(
            " - "
        )[0]

'01.01.2022 00:00'

In [45]:

time_resolution_number =1 
time_resolution_unit = 'minutes'
# Convert the selected time resolution to minutes
time_resolution_minutes = time_resolution_number * (
    60 if time_resolution_unit == "hours" else 1
)

# Set the 'time_column' as the index explicitly
df_read.set_index(time_column, inplace=True)

# Find the first row with '-'
# ENTSO-e puts "-" when data is missing for the future
# first_invalid_row = df_read.eq("-").any(axis=1).idxmax()
first_invalid_row_time = first_invalid_row.split(
            " - "
        )[0]


In [46]:
df_read.tail()

,Day-ahead Total Load Forecast [MW] - Netherlands (NL),Actual Total Load [MW] - Netherlands (NL)
Time (CET/CEST),,
2023-12-31 22:45:00,-,10085.0
2023-12-31 23:00:00,-,10085.0
2023-12-31 23:15:00,-,10085.0
2023-12-31 23:30:00,-,10085.0
2023-12-31 23:45:00,-,10085.0


In [51]:


first_invalid_row_time = pd.to_datetime(first_invalid_row_time, format='%d.%m.%Y %H:%M')

first_invalid_row_time

Timestamp('2023-11-05 13:45:00')

In [49]:

# find the dataset frequency
difference = df_read.index.to_series().diff()[1]
minutes_difference = difference.total_seconds() / 60


C:\Users\saeed.misaghian\AppData\Local\Temp\ipykernel_12540\684306115.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  difference = df_read.index.to_series().diff()[1]


In [50]:
minutes_difference

15.0

In [24]:
df_read

,Day-ahead Total Load Forecast [MW] - Netherlands (NL),Actual Total Load [MW] - Netherlands (NL)
Time (CET/CEST),,
2023-01-01 00:00:00,11330.0,9912.0
2023-01-01 00:15:00,11251.0,9813.0
2023-01-01 00:30:00,11154.0,9877.0
2023-01-01 00:45:00,11045.0,9861.0
2023-01-01 01:00:00,10929.0,9847.0
...,...,...
2023-12-31 22:45:00,10919.0,10085.0
2023-12-31 23:00:00,10919.0,10085.0
2023-12-31 23:15:00,10919.0,10085.0


In [53]:
first_invalid_row_time

Timestamp('2023-11-05 13:45:00')

In [54]:

# Keep only the rows until the row before the first_invalid_row
df_read = df_read.loc[
    : first_invalid_row_time - pd.Timedelta(minutes=minutes_difference)
]


In [55]:
df_read

,Day-ahead Total Load Forecast [MW] - Netherlands (NL),Actual Total Load [MW] - Netherlands (NL)
Time (CET/CEST),,
2023-01-01 00:00:00,11330,9912.0
2023-01-01 00:15:00,11251,9813.0
2023-01-01 00:30:00,11154,9877.0
2023-01-01 00:45:00,11045,9861.0
2023-01-01 01:00:00,10929,9847.0
...,...,...
2023-11-05 12:30:00,9501,10087.0
2023-11-05 12:45:00,9420,10034.0
2023-11-05 13:00:00,9491,10102.0


In [56]:

# Check for duplicate index values
duplicates = df_read.index.duplicated()

# Handle duplicates (for example, by keeping the first occurrence and dropping the others)
df_read = df_read[~duplicates]
# be sure that columns are not object (we have already taken care of nan etc.. so we should have only numbers)
df_read = df_read.apply(pd.to_numeric, errors="coerce")

df_read = df_read.resample(f"{time_resolution_minutes}T").interpolate()

In [57]:
df_read

,Day-ahead Total Load Forecast [MW] - Netherlands (NL),Actual Total Load [MW] - Netherlands (NL)
Time (CET/CEST),,
2023-01-01 00:00:00,11330.000000,9912.000000
2023-01-01 00:01:00,11324.733333,9905.400000
2023-01-01 00:02:00,11319.466667,9898.800000
2023-01-01 00:03:00,11314.200000,9892.200000
2023-01-01 00:04:00,11308.933333,9885.600000
...,...,...
2023-11-05 13:26:00,9631.666667,10082.066667
2023-11-05 13:27:00,9635.000000,10082.800000
2023-11-05 13:28:00,9638.333333,10083.533333


In [60]:
import datetime
date_of_interest = datetime.date(2023, 7, 6)
df_day_of_interest = df_read.loc[date_of_interest.strftime('%Y-%m-%d')]

In [61]:
df_day_of_interest

,Day-ahead Total Load Forecast [MW] - Netherlands (NL),Actual Total Load [MW] - Netherlands (NL)
Time (CET/CEST),,
2023-07-06 00:00:00,12890.0,11614.0
2023-07-06 00:01:00,12885.8,11600.8
2023-07-06 00:02:00,12881.6,11587.6
2023-07-06 00:03:00,12877.4,11574.4
2023-07-06 00:04:00,12873.2,11561.2
...,...,...
2023-07-06 23:55:00,12493.0,11762.0
2023-07-06 23:56:00,12488.4,11744.4
2023-07-06 23:57:00,12483.8,11726.8
